# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [25]:
weather_Output = pd.read_csv("weather_Output.csv", encoding="utf-8")
Cities = cities.dropna()
del Cities["Unnamed: 0"]
Cities.head(20)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,ribeira grande,20,ES,1586373114,82,42.75,-8.44,17.78,2.10
1,puerto ayora,100,ES,1586373114,55,36.59,-6.23,19.00,5.70
2,bredasdorp,99,ZA,1586372795,72,-34.53,20.04,16.11,6.20
3,rikitea,51,PF,1586372785,81,-23.12,-134.97,24.94,5.26
4,katsuura,0,JP,1586372840,76,35.13,140.30,13.00,4.10
5,grand gaube,1,US,1586373114,14,38.88,-109.50,20.00,2.60
6,hermanus,95,ZA,1586372784,73,-34.42,19.23,17.22,2.68
7,valparaiso,90,CL,1586372714,100,-33.04,-71.63,16.00,3.10
8,veraval,0,IN,1586373114,88,20.90,70.37,24.50,4.53
9,busselton,4,AU,1586372799,51,-33.65,115.33,16.11,3.72


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [26]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = Cities[["Lat", "Lng"]]

humidity = Cities["Humidity"].astype(float)


In [27]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [71]:
vacation_no_cloud = Cities[Cities["Cloudiness"] == 0]

# Filter vacation with max temp above 70 degrees F
vacation_above_70_degrees = vacation_no_cloud[vacation_no_cloud["Max Temp"] > 21]
# # Filter vacation with max temp below 80 degrees F
vacation_below_80_degrees = vacation_above_70_degrees[vacation_above_70_degrees["Max Temp"] < 26]
# # Filter vacation with wind speed below 10 mph
vacation_slow_wind = vacation_below_80_degrees[vacation_below_80_degrees["Wind Speed"] < 10]
# # Filter vacation with humidity below 60 %
perfect_vacation = vacation_slow_wind[vacation_slow_wind["Humidity"] < 60]
# # Set Index
indexed_perfect_vacation = perfect_vacation.reset_index()
# del indexed_perfect_vacation["index"]
indexed_perfect_vacation

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,106,valdivia,0,CL,1586372917,43,-39.81,-73.25,23.00,2.10
1,144,namibe,0,AO,1586372853,57,-15.20,12.15,23.75,6.73
2,252,luxor,0,EG,1586373143,33,25.70,32.64,24.00,2.10
3,323,moron,0,AR,1586373151,42,-34.65,-58.62,21.11,3.10
4,384,ancud,0,CL,1586372821,35,-41.87,-73.82,23.00,2.10
5,427,bati,0,ET,1586373163,24,11.18,40.02,22.48,0.57
6,513,viedma,0,AR,1586373173,20,-40.81,-63.00,22.19,5.86
7,518,kang,0,BW,1586373174,19,-23.68,22.79,23.28,0.11


In [87]:
vaca_locations = indexed_perfect_vacation[["Lat", "Lng"]]

vaca_humidity = indexed_perfect_vacation["Humidity"].astype(float)

# Plot Heatmap
vaca_fig = gmaps.figure()

# Create heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vaca_fig.add_layer(vaca_heat_layer)

plt.savefig("vaca_fig")

# Display figure
vaca_fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [84]:
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(indexed_perfect_vacation["City"])):

    lat = indexed_perfect_vacation.loc[city]["Lat"]
    lng = indexed_perfect_vacation.loc[city]["Lng"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
indexed_perfect_vacation["Hotel"] = hotels
narrowed_city_df = indexed_perfect_vacation
hotel_df = indexed_perfect_vacation

In [85]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [86]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
vaca_fig.add_layer(markers)

# Display Map
vaca_fig

Figure(layout=FigureLayout(height='420px'))